In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from ml_experiments.analyze import get_df_runs_from_mlflow_sql, get_missing_entries, get_common_combinations, get_df_with_combinations
import plotly.express as px
from itertools import product
import time
import numpy as np

# Save Results

## Load mlflow runs

In [2]:
db_port = 5003
db_name = 'recursive_clustering'
w = 'clust9'
# url = f'postgresql://{w}.ceremade.dauphine.lan:{db_port}/{db_name}'
url = f'postgresql://belucci@localhost:{db_port}/{db_name}'
engine = create_engine(url)
query = 'SELECT experiments.name from experiments'
experiment_names = pd.read_sql(query, engine)['name'].tolist()
# results_dir = Path('~/tab_benchmark/results')
# os.makedirs(results_dir, exist_ok=True)

In [3]:
experiment_names

['Default',
 'blob_experiment',
 'hpo_classif_experiment',
 'hpo_openml_experiment',
 'hpo_gaussian_experiment',
 'hpo_openml_final',
 'outlier_hc',
 'time_hc']

In [4]:
experiments_names = [
    'hpo_openml_final',
]

In [5]:
params_columns = [
    'model_nickname',
    'n_samples',
    'n_features',
    'n_classes',
    'seed_model',
    'dataset_id',
    'dataset_name',
    # 'model_params/normalization',
    'direction',
    'hpo_metric',
    'standardize'
]

In [6]:
latest_metrics_columns = [
    'fit_model_return_elapsed_time',
    'max_memory_used',
    'n_clusters_',
    'rand_score',
    'adjusted_rand',
    'mutual_info',
    'adjusted_mutual_info',
    'normalized_mutual_info',
    'homogeneity',
    'completeness',
    'v_measure',
    'silhouette',
    'calinski_harabasz_score',
    'davies_bouldin_score',
    'inertia_score',
    'best_n_clusters_',
    'best_rand_score',
    'best_adjusted_rand',
    'best_mutual_info',
    'best_adjusted_mutual_info',
    'best_normalized_mutual_info',
    'best_homogeneity_completeness_v_measure',
    'best_silhouette',
    'best_calinski_harabasz_score',
    'best_davies_bouldin_score',
    'best_inertia_score',
    'best_homogeneity',
    'best_completeness',
    'best_v_measure',
]

In [7]:
tags_columns = [
    'raised_exception',
    'EXCEPTION',
    'parent_run_id',
    'best_child_run_id',
]

In [8]:
# parameters of each model, they are saved as tags for the parent run
parameters = {
    'RecursiveClustering': ['components_size', 'repetitions', 'kmeans_n_clusters'],
    'KMeans': ['n_clusters'],
    'HDBSCAN': ['min_cluster_size'],
    'DBSCAN': ['eps', 'min_samples'],
    'AffinityPropagation': ['damping'],
    'OPTICS': ['min_samples'],
    'SpectralSubspaceRandomization': ['n_similarities', 'sampling_ratio', 'sc_n_clusters'],
    'WardAgglomerativeClustering': ['n_clusters'],
}
all_model_parameters = [p for model, params in parameters.items() for p in params]
all_model_parameters = list(set(all_model_parameters))
for param in all_model_parameters:
    tags_columns.append(param)

In [9]:
runs_columns = ['run_uuid', 'status', 'start_time', 'end_time']
experiments_columns = []
other_table = 'params'
other_table_keys = params_columns
df_params = get_df_runs_from_mlflow_sql(engine, runs_columns=runs_columns, experiments_columns=experiments_columns, experiments_names=experiments_names, other_table=other_table, other_table_keys=other_table_keys)
df_latest_metrics = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='latest_metrics', other_table_keys=latest_metrics_columns)
df_tags = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='tags', other_table_keys=tags_columns)

In [10]:
df_runs_raw = df_params.join(df_latest_metrics)
df_runs_raw = df_runs_raw.join(df_tags)

In [11]:
df_runs_raw_parents = df_runs_raw.copy()
df_runs_raw_parents = df_runs_raw_parents.loc[df_runs_raw_parents['parent_run_id'].isna()]

In [12]:
# get all metrics beside best or last
metrics_columns = [
    'adjusted_rand',
    'silhouette',
    'calinski_harabasz_score',
    'davies_bouldin_score',
    'inertia_score',
]
metrics_columns_query = [f"'{m}'" for m in metrics_columns]
metrics_columns_query = ', '.join(metrics_columns_query)
experiments_names_query = [f"'{name}'" for name in experiments_names]
experiments_names_query = ', '.join(experiments_names_query)
query = f"""
SELECT
	runs.run_uuid,
	metrics."key",
	metrics.value,
	metrics.step 
FROM
	runs
LEFT JOIN metrics ON
	metrics.run_uuid = runs.run_uuid
LEFT JOIN experiments ON 
	experiments.experiment_id = runs.experiment_id  
WHERE 
	metrics."key" IN ({metrics_columns_query})
	AND experiments."name" IN ({experiments_names_query})
"""
df_metrics = pd.read_sql(query, engine)
df_metrics = df_metrics.pivot_table(index=['run_uuid', 'step'], columns='key', values='value')
df_metrics

,key,adjusted_rand,calinski_harabasz_score,davies_bouldin_score,inertia_score,silhouette
run_uuid,step,,,,,
000109ef351e4cf7a088ad85ad7e0643,0,0.000000,-1.000000,1000.000000,1.279360e+05,-1.000000
00032b51730847ebb72e762407701b9b,0,0.082347,558.383684,1.100218,9.153871e+03,0.371600
00058d452fdd4128810e0b4bbf070246,0,0.113154,71.531337,1.431907,4.009946e+03,0.192575
0008dd178961435e884e90714ac29054,0,0.146457,4.302437,5.825635,9.054871e+05,-0.150969
001070aa390c40369818a820ccda66c3,0,0.140046,8.696977,3.885626,3.376744e+05,-0.051281
...,...,...,...,...,...,...
fff60e5db53e4316bde81a7a5e0aa3c5,0,0.347694,35.930376,3.407080,3.757653e+04,0.051182
fff7d9ffe83c41eb9c296012f06989cc,0,0.356844,16.279657,2.048616,2.497868e+05,-0.051084
fff95330848845f3a40af5ff9487fce9,0,-0.011991,79.069922,2.133947,4.063221e+05,-0.143267


In [13]:
df_runs_raw_parents

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,kmeans_n_clusters,min_cluster_size,min_samples,n_clusters,n_similarities,parent_run_id,raised_exception,repetitions,sampling_ratio,sc_n_clusters
run_uuid,,,,,,,,,,,,,,,,,,,,,
0018714386f44f95ab7141d480ce09f0,FINISHED,1737557036103,1.737557e+12,1044,eye_movements,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,NaN,10,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN
00502ff54da4466191d5fadd2ab0f945,FINISHED,1737556950053,1.737557e+12,7,audiology,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,NaN,5,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN
00b6f8e09ce1419e84214bb9734a9bff,FINISHED,1737589285408,1.737590e+12,23380,cjs,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,NaN,3,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN
00b7b74b49c2418e8cd9108326d76f54,FINISHED,1737504019181,1.737512e+12,46335,primary-tumor_clean,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,8,NaN,NaN,NaN,NaN,NaN,False,6,NaN,NaN
00be057a1db84ac1b41fd40f1cbc4889,FAILED,1737643267392,1.737643e+12,1567,poker-hand,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fdb13063ebc24ee887da97c3d7b67523,FINISHED,1737643758563,1.737646e+12,458,analcatdata_authorship,maximize,adjusted_rand,OPTICS,NaN,NaN,...,NaN,NaN,2,NaN,NaN,NaN,False,NaN,NaN,NaN
fdc6bb9a534743c7be6ce026fd67252a,FINISHED,1737556999587,1.737557e+12,61,iris,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,NaN,NaN,5,NaN,NaN,NaN,False,NaN,NaN,NaN
feca472c0d2149368332abf98bab6bd7,FINISHED,1737516203798,1.737519e+12,42,soybean,maximize,adjusted_rand,AffinityPropagation,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN


## Delete duplicate runs (if any) and complete some models that cannot run with some datasets

In [14]:
non_duplicate_columns = [
    'model_nickname',
    'dataset_id',
    # 'model_params/normalization',
    'direction',
    'hpo_metric',
    'standardize',
]
df_runs_parents = df_runs_raw_parents.dropna(axis=0, how='all', subset=['best_adjusted_rand']).copy()
df_runs_parents = df_runs_parents.loc[(~df_runs_parents.duplicated(non_duplicate_columns))]

no_run_time = 4*3600
df_runs_timed_out = df_runs_raw.copy()
df_runs_timed_out = df_runs_timed_out.loc[df_runs_timed_out.EXCEPTION == 'FunctionTimedOut']
df_runs_timed_out = df_runs_timed_out.loc[(~df_runs_timed_out.duplicated(non_duplicate_columns))]
df_runs_timed_out['fit_model_return_elapsed_time'] = no_run_time
df_runs = pd.concat([df_runs, df_runs_timed_out])

status = 'FAILED'
no_run_memory = 2*120000
raised_exception = True
EXCEPTION = 'NoRun'
start_time = time.time() * 1000
end_time = time.time() * 1000

# IRFLLRR do not run with n_samples > 14427
model_nickname = 'IRFLLRR'
n_samples = ['14427', '50000']
n_features = ['100', '347', '1202', '4163', '14427', '50000']
combinations = product(n_samples, n_features)
df_cat = []
indexes = []
for n_sample, n_feature in combinations:
    df = {}
    index = f'{model_nickname}_{n_sample}_{n_feature}'
    df['status'] = status
    df['start_time'] = start_time
    df['end_time'] = end_time
    df['model_nickname'] = model_nickname
    df['n_features'] = n_feature
    df['n_samples'] = n_sample
    df['fit_model_return_elapsed_time'] = no_run_time
    df['max_memory_used'] = no_run_memory
    df['EXCEPTION'] = EXCEPTION
    df['raised_exception'] = raised_exception
    df_cat.append(df)
    indexes.append(index)
df_no_run = pd.DataFrame(df_cat, index=indexes)
df_runs = pd.concat([df_runs, df_no_run])

# KMeansProj do not run with (14427,14427), (14427,50000), (50000,14427), (50000,50000)
model_nickname = 'KMeansProj'
n_samples = ['14427', '50000']
n_features = ['14427', '50000']
combinations = product(n_samples, n_features)
df_cat = []
indexes = []
for n_sample, n_feature in combinations:
    df = {}
    index = f'{model_nickname}_{n_sample}_{n_feature}'
    df['status'] = status
    df['start_time'] = start_time
    df['end_time'] = end_time
    df['model_nickname'] = model_nickname
    df['n_features'] = n_feature
    df['n_samples'] = n_sample
    df['fit_model_return_elapsed_time'] = no_run_time
    df['max_memory_used'] = no_run_memory
    df['EXCEPTION'] = EXCEPTION
    df['raised_exception'] = raised_exception
    df_cat.append(df)
    indexes.append(index)
df_no_run = pd.DataFrame(df_cat, index=indexes)
df_runs = pd.concat([df_runs, df_no_run])

# HDBSCAN do not run with (14427,14427), (14427,50000), and n_samples = 50000 n_features > 347
model_nickname = 'HDBSCAN'
n_samples = ['50000']
n_features = ['347', '1202', '4163', '14427', '50000']
combinations = list(product(n_samples, n_features))
combinations += [('14427', '14427'), ('14427', '50000')]
df_cat = []
indexes = []
for n_sample, n_feature in combinations:
    df = {}
    index = f'{model_nickname}_{n_sample}_{n_feature}'
    df['status'] = status
    df['start_time'] = start_time
    df['end_time'] = end_time
    df['model_nickname'] = model_nickname
    df['n_features'] = n_feature
    df['n_samples'] = n_sample
    df['fit_model_return_elapsed_time'] = no_run_time
    df['max_memory_used'] = no_run_memory
    df['EXCEPTION'] = EXCEPTION
    df['raised_exception'] = raised_exception
    df_cat.append(df)
    indexes.append(index)
df_no_run = pd.DataFrame(df_cat, index=indexes)
df_runs = pd.concat([df_runs, df_no_run])

# SpectralSubspaceRandomization do not run with n_samples > 50000
model_nickname = 'SpectralSubspaceRandomization'
n_samples = ['50000']
n_features = ['100', '347', '1202', '4163', '14427', '50000']
combinations = list(product(n_samples, n_features))
df_cat = []
indexes = []
for n_sample, n_feature in combinations:
    df = {}
    index = f'{model_nickname}_{n_sample}_{n_feature}'
    df['status'] = status
    df['start_time'] = start_time
    df['end_time'] = end_time
    df['model_nickname'] = model_nickname
    df['n_features'] = n_feature
    df['n_samples'] = n_sample
    df['fit_model_return_elapsed_time'] = no_run_time
    df['max_memory_used'] = no_run_memory
    df['EXCEPTION'] = EXCEPTION
    df['raised_exception'] = raised_exception
    df_cat.append(df)
    indexes.append(index)
df_no_run = pd.DataFrame(df_cat, index=indexes)
df_runs = pd.concat([df_runs, df_no_run])

# ensure no duplicates
df_runs = df_runs.loc[(~df_runs.duplicated(non_duplicate_columns))]

# Missing

In [15]:
df_runs_parents['model_nickname'].unique().tolist()

['HDBSCAN',
 'RecursiveClustering',
 'OPTICS',
 'SpectralSubspaceRandomization',
 'DBSCAN',
 'WardAgglomerativeClustering',
 'KMeans',
 'AffinityPropagation']

In [16]:
non_duplicate_columns = [
    'model_nickname',
    'dataset_id',
    # 'model_params/normalization',
    'direction',
    'hpo_metric',
    'standardize',
]

In [17]:
datasets = pd.read_csv('/home/bbelucci/code/recursive_clustering/recursive_clustering/openml_datasets.csv')

In [18]:
dataset_id = [40685, 39, 61, 182, 40984, 1478, 1568]
df = datasets.loc[datasets['dataset_id'].isin(dataset_id)].copy()
df['n_categorical_features'] = df['n_categorical_features'] - 1  # remove the target column
df['n_instances'] = df['n_instances'].astype(int)
df['n_features'] = df['n_features'].astype(int)
df['n_classes'] = df['n_classes'].astype(int)
df['n_categorical_features'] = df['n_categorical_features'].astype(int)
df = df[['dataset_name', 'dataset_id', 'n_instances', 'n_features', 'n_categorical_features', 'n_classes', 'task_id']]
# df = df.rename(columns={'dataset_name': 'Dataset', 'dataset_id': 'OpenML ID', 'n_instances': 'N. Samples', 'n_features': 'N. Features', 'n_classes': 'N. Classes', 'n_categorical_features': 'N. Categorical Features'})
df = df.rename(columns={'dataset_name': 'Dataset', 'dataset_id': 'OpenML ID', 'n_instances': '$\samplesize$', 'n_features': '$\dimsize$', 'n_classes': '$\clusternum$', 'n_categorical_features': '$\categoricalnum$'})

In [19]:
df

,Dataset,OpenML ID,$\samplesize$,$\dimsize$,$\categoricalnum$,$\clusternum$,task_id
26,ecoli,39,336,8,0,8,145977
34,har,1478,10299,562,0,6,14970
38,iris,61,150,5,0,3,59
55,nursery,1568,12958,9,8,4,9892
62,satimage,182,6430,37,0,6,2074
63,segment,40984,2310,20,0,7,146822
65,shuttle,40685,58000,10,0,7,146212


In [20]:
print(df.style.hide().to_latex(hrules=True))

\begin{tabular}{lrrrrrr}
\toprule
Dataset & OpenML ID & $\samplesize$ & $\dimsize$ & $\categoricalnum$ & $\clusternum$ & task_id \\
\midrule
ecoli & 39 & 336 & 8 & 0 & 8 & 145977 \\
har & 1478 & 10299 & 562 & 0 & 6 & 14970 \\
iris & 61 & 150 & 5 & 0 & 3 & 59 \\
nursery & 1568 & 12958 & 9 & 8 & 4 & 9892 \\
satimage & 182 & 6430 & 37 & 0 & 6 & 2074 \\
segment & 40984 & 2310 & 20 & 0 & 7 & 146822 \\
shuttle & 40685 & 58000 & 10 & 0 & 7 & 146212 \\
\bottomrule
\end{tabular}



In [21]:
dataset_id = datasets['dataset_id'].tolist()
# too long datasets 42855, 1596, 40927, 40996, 41168, 1483, 1459, 554
dataset_id = [40685, 39, 61, 182, 40984, 1478, 1568]
dataset_id = [str(d) for d in dataset_id if d not in [42855, 1596, 40927, 40996, 41168, 1483, 1459, 554, 1567, 41165, 1509, 40668, 1118]]
direction = ['maximize']
metric = ['adjusted_rand']
standardize = ['True']
model_nickname = [
    'AffinityPropagation',
    'HDBSCAN',
    'WardAgglomerativeClustering',
    'SpectralSubspaceRandomization',
    'RecursiveClustering',
    'KMeans',
    'DBSCAN',
    'OPTICS',
]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, dataset_id, direction, metric, standardize]
df_missing = get_missing_entries(df_runs_parents, columns_names, should_contain_values)

In [22]:
df_missing

,model_nickname,dataset_id,direction,hpo_metric,standardize
0,AffinityPropagation,40685,maximize,adjusted_rand,True
1,SpectralSubspaceRandomization,40685,maximize,adjusted_rand,True
2,SpectralSubspaceRandomization,1568,maximize,adjusted_rand,True


In [23]:
print(*df_missing['dataset_id'].unique().tolist())

40685 1568


In [24]:
df = df_runs_raw_parents.copy()
df = df.loc[df['model_nickname'] == 'DBSCAN']
df = df.loc[df['dataset_id'] == '1459']
df

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,kmeans_n_clusters,min_cluster_size,min_samples,n_clusters,n_similarities,parent_run_id,raised_exception,repetitions,sampling_ratio,sc_n_clusters
run_uuid,,,,,,,,,,,,,,,,,,,,,
4dbb7e53d52f46548b57fee10b285c21,FINISHED,1737589030899,1.737589e+12,1459,artificial-characters,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,NaN,NaN,7,NaN,NaN,NaN,False,NaN,NaN,NaN


In [25]:
df = df_runs_parents.loc[df_runs_parents['model_nickname'] == 'DBSCAN']
df = df.loc[df['dataset_id'] == '1118']
df

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,kmeans_n_clusters,min_cluster_size,min_samples,n_clusters,n_similarities,parent_run_id,raised_exception,repetitions,sampling_ratio,sc_n_clusters
run_uuid,,,,,,,,,,,,,,,,,,,,,
ea59887cdd6f470189caf89611190b5c,FINISHED,1737589058694,1.737590e+12,1118,chess,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,NaN,NaN,5,NaN,NaN,NaN,False,NaN,NaN,NaN


# Get common combinations

In [84]:
model_nickname = [
    # 'KMeansProj',
    # 'IRFLLRR',
    # 'Clique',
    'HDBSCAN',
    'SpectralSubspaceRandomization',
    'SingleAgglomerativeClustering',
    'SpectralClustering',
    'RecursiveClustering',
    'OPTICS',
    'Proclus',
    'WardAgglomerativeClustering',
    'KMeans',
    'DBSCAN',
    'CompleteAgglomerativeClustering',
    'AverageAgglomerativeClustering',
    'AffinityPropagation'
]
df = df_runs_parents.copy()
df = df.loc[df['model_nickname'].isin(model_nickname)]
column = 'model_nickname'
combination_columns = [
    'n_samples',
    'n_features',
    'pct_random',
    'seed_unified',
    'class_sep',
]
common_combinations = get_common_combinations(df, column, combination_columns)

In [85]:
df_common = get_df_with_combinations(df, combination_columns, common_combinations)

In [86]:
df_common['n_samples'] = df_common['n_samples'].astype(int)
df_common['n_features'] = df_common['n_features'].astype(int)
df_common['pct_random'] = df_common['pct_random'].astype(float)
df_common['class_sep'] = df_common['class_sep'].astype(float)
df_common['seed_unified'] = df_common['seed_unified'].astype(int)
df_common['n_classes'] = df_common['n_classes'].astype(int)

In [87]:
df_common

key,status,start_time,end_time,class_sep,model_nickname,n_classes,n_features,n_informative,n_random,n_samples,...,max_memory_used,mutual_info,n_clusters_,normalized_mutual_info,rand_score,silhouette,EXCEPTION,best_child_run_id,parent_run_id,raised_exception
run_uuid,,,,,,,,,,,,,,,,,,,,,
0018167e4ff1486a8a909f25fdaa5d76,FINISHED,1734289233443,1.734364e+12,100.0,RecursiveClustering,5,100,2,16,1000,...,2077.516,1.609438,5.0,1.000000,1.000000,0.936638,NaN,0fc46cf1ca434f86841b195a2b92e37d,NaN,False
002193f658094383b0f627c470ba56ae,FINISHED,1734314061235,1.734333e+12,100.0,SpectralSubspaceRandomization,5,100,2,16,100,...,412.472,1.609438,7.0,0.921757,0.960404,0.615130,NaN,bedee9589bea483ca7379588f1db31b8,NaN,False
0023abb5ee2140a7b29b46776f01d1cb,FINISHED,1734334144613,1.734467e+12,10.0,Proclus,5,10000,2,16,1000,...,656.852,0.017358,8.0,0.010126,0.686326,-0.034658,NaN,d75026e4d65a4841b4f3efb2cee88fad,NaN,False
002ff3e2dd904189a04b1568c2a4cfcb,FINISHED,1734288888094,1.734326e+12,10.0,WardAgglomerativeClustering,5,100,2,16,1000,...,2062.552,1.609438,12.0,0.800623,0.895393,0.105404,NaN,7009338b4aac4a1b81815335fffe138d,NaN,False
003d9c7cb27d4db18cfd570de01c35dd,FINISHED,1734292682676,1.734321e+12,100.0,KMeans,5,1000,2,16,1000,...,2069.860,1.609438,5.0,1.000000,1.000000,0.881070,NaN,f4587824429f4f848e6be406124a2de5,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffbdad9588f0455480ca59645773da0f,FINISHED,1734295865271,1.734410e+12,100.0,AverageAgglomerativeClustering,5,1000,2,16,1000,...,2141.648,1.609438,5.0,1.000000,1.000000,0.787634,NaN,5e6823c34f3749a3832cb34c2cde2b52,NaN,False
ffbea3f03bf94cb89131832f6d2df628,FINISHED,1734290981889,1.734316e+12,10.0,KMeans,5,1000,2,16,1000,...,2065.080,0.784140,10.0,0.402668,0.812697,0.064731,NaN,b0dddd5d266048b7baaa2f5b7b7bce5f,NaN,False
ffd07194432a4f399e2677dec7e98d62,FINISHED,1734299196992,1.734435e+12,100.0,SingleAgglomerativeClustering,5,1000,2,16,100,...,2137.760,1.609438,7.0,0.968492,0.988889,0.563874,NaN,9cf711ec5db346c29a64c10a156f825d,NaN,False


# Plots

In [26]:
datasets = pd.read_csv('/home/bbelucci/code/recursive_clustering/recursive_clustering/openml_datasets.csv')

In [27]:
df = df_runs_parents.copy()
df['dataset_id'] = df['dataset_id'].astype(int)
df = df.join(datasets.set_index('dataset_id')[['n_instances', 'n_features', 'n_classes']], on='dataset_id', rsuffix='_dataset')

In [28]:
df = df.loc[df['model_nickname'] == 'RecursiveClustering']
df = df.sort_values('best_adjusted_rand', ascending=False)

In [29]:
df[['dataset_id', 'dataset_name', 'n_instances', 'n_features_dataset', 'n_classes_dataset', 'best_adjusted_rand']]

,dataset_id,dataset_name,n_instances,n_features_dataset,n_classes_dataset,best_adjusted_rand
run_uuid,,,,,,
da0fe18112cf4676b2d2e58c6fc2ad28,458,analcatdata_authorship,841.0,71.0,4,0.873840
10ce6db39cd94cce8804face72448da4,1466,cardiotocography,2126.0,36.0,10,0.813451
74a2b0d1e1cc414383e6feba490704ca,39,ecoli,336.0,8.0,8,0.758326
698087e04b1b4eb5bf54caf1e4d628f4,377,synthetic_control,600.0,61.0,6,0.667671
98c93373a4224f4cb4315e6359715842,40685,shuttle,58000.0,10.0,7,0.651919
...,...,...,...,...,...,...
437497276aa54cd3b7b4c41b17a03f42,23,cmc,1473.0,10.0,3,0.029548
df7d45c2bdd64ca2853738f487bac5eb,40927,CIFAR_10,60000.0,3073.0,10,0.014253
562841d9791d44a9a7e5422ec3ff2eae,469,analcatdata_dmft,797.0,5.0,6,0.013220


In [114]:
def get_parameters_string(row):
    parameter_names = {
        'components_size': 'q',
        'repetitions': 'R',
        'kmeans_n_clusters': 'C',
        'n_clusters': 'C',
        'min_cluster_size': 'C_{\\text{min}}',
        'eps': '\epsilon',
        'min_samples': 'n_{\\text{min}}',
        'damping': '\lambda',
        'n_similarities': 'm',
        'sampling_ratio': 'r',
        'sc_n_clusters': 'C',
    }
    first = True
    str = ''
    for p in all_model_parameters:
        if not pd.isna(row[p]):
            if not first:
                str += '; '
            else:
                first = False
            value = float(row[p])
            if value.is_integer():
                value = int(value)
                str += f'${parameter_names[p]}={value}$'
            else:
                str += f'${parameter_names[p]}={value:0.2f}$'
    return str        

In [115]:
def highlight_max(df):
    max_values = df.groupby(level=0).transform('max')
    is_highlighted = df == max_values
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ''
    df_css[is_highlighted] = 'font-weight: bold'
    return df_css

In [116]:
datasets_ids = [40685, 39, 61, 182, 40984, 1478, 1568]
datasets_ids = [str(d) for d in datasets_ids]
models_nicknames = ['RecursiveClustering', 'KMeans', 'HDBSCAN', 'DBSCAN', 'AffinityPropagation', 'OPTICS', 'SpectralSubspaceRandomization', 'WardAgglomerativeClustering']
df = df_runs_parents.copy()
df = df.loc[df['dataset_id'].isin(datasets_ids)]
df = df.loc[df['model_nickname'].isin(models_nicknames)]
df['Parameters'] = df.apply(get_parameters_string, axis=1)
df = df[['model_nickname', 'dataset_name', 'best_adjusted_rand', 'Parameters']]
models_names = {
    'RecursiveClustering': 'CoHiRF',
    'KMeans': 'K-Means',
    'HDBSCAN': 'HDBSCAN',
    'DBSCAN': 'DBSCAN',
    'AffinityPropagation': 'Affinity Propagation',
    'OPTICS': 'OPTICS',
    'SpectralSubspaceRandomization': 'SC-SRGF',
    'WardAgglomerativeClustering': "Ward's Method",
}
parameters = {
    'RecursiveClustering': ['components_size', 'repetitions', 'kmeans_n_clusters'],
    'KMeans': ['n_clusters'],
    'HDBSCAN': ['min_cluster_size'],
    'DBSCAN': ['eps', 'min_samples'],
    'AffinityPropagation': ['damping'],
    'OPTICS': ['min_samples'],
    'SpectralSubspaceRandomization': ['n_similarities', 'sampling_ratio', 'sc_n_clusters'],
    'WardAgglomerativeClustering': ['n_clusters'],
}
df = df.replace({'model_nickname': models_names})
df = df.rename(columns={'best_adjusted_rand': 'ARI', 'model_nickname': 'Model', 'dataset_name': 'Dataset'})
df = df.groupby(['Dataset', 'Model']).agg({'ARI': 'mean', 'Parameters': 'first'})

In [117]:
df

key                                 ARI                         Parameters
Dataset  Model                                                            
ecoli    Affinity Propagation  0.248155                      $\delta=0.58$
         CoHiRF                0.758326              $C=7$; $R=10$; $q=11$
         DBSCAN                0.345131       $n_{min}=7$; $\epsilon=0.78$
         HDBSCAN               0.397572                       $C_{min}=10$
         K-Means               0.718701                              $C=6$
         OPTICS                0.313694                       $n_{min}=10$
         SC-SRGF               0.723193   $\rho=0.80$; $n_{sim}=15$; $C=4$
         Ward's Method         0.734778                              $C=7$
har      Affinity Propagation  0.312771                      $\delta=1.00$
         CoHiRF                0.491249               $C=4$; $R=8$; $q=11$
         DBSCAN                0.302146      $n_{min}=3$; $\epsilon=13.91$
         HDBSCAN               0.286958                        $C_{min}=6$
         K-Means               0.438310                              $C=9$
         OPTICS                0.000694                        $n_{min}=4$
         SC-SRGF               0.545981  $\rho=0.45$; $n_{sim}=21$; $C=20$
         Ward's Method         0.510766                              $C=4$
iris     Affinity Propagation  0.477171                      $\delta=0.98$
         CoHiRF                0.631048               $C=4$; $R=7$; $q=26$
         DBSCAN                0.558371       $n_{min}=5$; $\epsilon=1.37$
         HDBSCAN               0.563751                        $C_{min}=9$
         K-Means               0.592333                              $C=3$
         OPTICS                0.396136                       $n_{min}=10$
         SC-SRGF               0.865256   $\rho=0.79$; $n_{sim}=26$; $C=3$
         Ward's Method         0.615323                              $C=3$
nursery  Affinity Propagation  0.015693                      $\delta=0.95$
         CoHiRF                0.440254               $C=3$; $R=6$; $q=30$
         DBSCAN                0.000000       $n_{min}=5$; $\epsilon=0.50$
         HDBSCAN               0.014181                        $C_{min}=9$
         K-Means               0.150396                              $C=2$
         OPTICS                0.000000                        $n_{min}=9$
         Ward's Method         0.253798                              $C=3$
satimage Affinity Propagation  0.190230                      $\delta=0.99$
         CoHiRF                0.583496              $C=9$; $R=10$; $q=19$
         DBSCAN                0.296792       $n_{min}=7$; $\epsilon=1.54$
         HDBSCAN               0.308123                        $C_{min}=6$
         K-Means               0.566324                              $C=7$
         OPTICS                0.022515                        $n_{min}=2$
         SC-SRGF               0.618338   $\rho=0.76$; $n_{sim}=21$; $C=4$
         Ward's Method         0.484501                              $C=5$
segment  Affinity Propagation  0.279673                      $\delta=0.93$
         CoHiRF                0.539809              $C=5$; $R=10$; $q=21$
         DBSCAN                0.250797       $n_{min}=7$; $\epsilon=0.54$
         HDBSCAN               0.390071                        $C_{min}=5$
         K-Means               0.512024                             $C=10$
         OPTICS                0.097270                       $n_{min}=10$
         SC-SRGF               0.481409  $\rho=0.65$; $n_{sim}=23$; $C=14$
         Ward's Method         0.446081                              $C=8$
shuttle  CoHiRF                0.651919                $C=6$; $R=8$; $q=5$
         DBSCAN                0.685962      $n_{min}=10$; $\epsilon=0.27$
         HDBSCAN               0.001018                        $C_{min}=2$
         K-Means               0.608404                              $C=2$
         OPTICS          

In [118]:
print(df.style.apply(highlight_max, subset='ARI', axis=None).format(precision=3, na_rep='No Run').to_latex(hrules=True, clines='skip-last;data', convert_css=True, column_format='p{1cm}lp{0.5cm}l'))

\begin{tabular}{p{1cm}lp{0.5cm}l}
\toprule
 & key & ARI & Parameters \\
Dataset & Model &  &  \\
\midrule
\multirow[c]{8}{*}{ecoli} & Affinity Propagation & 0.248 & $\delta=0.58$ \\
 & CoHiRF & \bfseries 0.758 & $C=7$; $R=10$; $q=11$ \\
 & DBSCAN & 0.345 & $n_{min}=7$; $\epsilon=0.78$ \\
 & HDBSCAN & 0.398 & $C_{min}=10$ \\
 & K-Means & 0.719 & $C=6$ \\
 & OPTICS & 0.314 & $n_{min}=10$ \\
 & SC-SRGF & 0.723 & $\rho=0.80$; $n_{sim}=15$; $C=4$ \\
 & Ward's Method & 0.735 & $C=7$ \\
\cline{1-4}
\multirow[c]{8}{*}{har} & Affinity Propagation & 0.313 & $\delta=1.00$ \\
 & CoHiRF & 0.491 & $C=4$; $R=8$; $q=11$ \\
 & DBSCAN & 0.302 & $n_{min}=3$; $\epsilon=13.91$ \\
 & HDBSCAN & 0.287 & $C_{min}=6$ \\
 & K-Means & 0.438 & $C=9$ \\
 & OPTICS & 0.001 & $n_{min}=4$ \\
 & SC-SRGF & \bfseries 0.546 & $\rho=0.45$; $n_{sim}=21$; $C=20$ \\
 & Ward's Method & 0.511 & $C=4$ \\
\cline{1-4}
\multirow[c]{8}{*}{iris} & Affinity Propagation & 0.477 & $\delta=0.98$ \\
 & CoHiRF & 0.631 & $C=4$; $R=7$; $q=26$

In [80]:
df

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,min_cluster_size,min_samples,n_clusters,n_similarities,parent_run_id,raised_exception,repetitions,sampling_ratio,sc_n_clusters,parameters
run_uuid,,,,,,,,,,,,,,,,,,,,,
0d3f0312fdfb4b4eb909bcaf25b1703e,FINISHED,1737516206511,1.737518e+12,40984,segment,maximize,adjusted_rand,AffinityPropagation,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,damping: 0.9294026686605588
10ac2ec99d13499a8b4d80afbbbe05a3,FINISHED,1737568999692,1.737569e+12,1568,nursery,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,NaN,3,NaN,NaN,False,NaN,NaN,NaN,n_clusters: 3
1577a524c14344f7a86903084f02949e,FINISHED,1737504126260,1.737504e+12,40984,segment,maximize,adjusted_rand,KMeans,NaN,NaN,...,NaN,NaN,10,NaN,NaN,False,NaN,NaN,NaN,n_clusters: 10
1c48c1907f1447e89c724333a731af26,FINISHED,1737504067203,1.737515e+12,1478,har,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,8,NaN,NaN,"components_size: 11, kmeans_n_clusters: 4, rep..."
1c8422d391974a038aaea93cffd7c20f,FINISHED,1737569000346,1.737569e+12,40984,segment,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,NaN,8,NaN,NaN,False,NaN,NaN,NaN,n_clusters: 8
27b16ca431d14bf295265e04a20572f0,FINISHED,1737557104228,1.737588e+12,40984,segment,maximize,adjusted_rand,SpectralSubspaceRandomization,NaN,NaN,...,NaN,NaN,NaN,23,NaN,False,NaN,0.6533337717772364,14,"n_similarities: 23, sampling_ratio: 0.65333377..."
294b4441caa04a7587a00051e738b19a,FINISHED,1737557013902,1.737557e+12,40984,segment,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,NaN,7,NaN,NaN,NaN,False,NaN,NaN,NaN,"eps: 0.5374182452826163, min_samples: 7"
2fcb935e2cb0474dafc839d7069548f9,FINISHED,1737556959605,1.737558e+12,182,satimage,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,6,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,min_cluster_size: 6
39be46ef4bcc4230b354dd8e14c8bffb,FINISHED,1737675145917,1.737676e+12,40685,shuttle,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,2,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,min_cluster_size: 2


In [50]:
df

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,min_cluster_size,min_samples,n_clusters,n_similarities,parent_run_id,raised_exception,repetitions,sampling_ratio,sc_n_clusters,parameters
run_uuid,,,,,,,,,,,,,,,,,,,,,
0d3f0312fdfb4b4eb909bcaf25b1703e,FINISHED,1737516206511,1.737518e+12,40984,segment,maximize,adjusted_rand,AffinityPropagation,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,
10ac2ec99d13499a8b4d80afbbbe05a3,FINISHED,1737568999692,1.737569e+12,1568,nursery,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,NaN,3,NaN,NaN,False,NaN,NaN,NaN,
1577a524c14344f7a86903084f02949e,FINISHED,1737504126260,1.737504e+12,40984,segment,maximize,adjusted_rand,KMeans,NaN,NaN,...,NaN,NaN,10,NaN,NaN,False,NaN,NaN,NaN,
1c48c1907f1447e89c724333a731af26,FINISHED,1737504067203,1.737515e+12,1478,har,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,8,NaN,NaN,
1c8422d391974a038aaea93cffd7c20f,FINISHED,1737569000346,1.737569e+12,40984,segment,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,NaN,8,NaN,NaN,False,NaN,NaN,NaN,
27b16ca431d14bf295265e04a20572f0,FINISHED,1737557104228,1.737588e+12,40984,segment,maximize,adjusted_rand,SpectralSubspaceRandomization,NaN,NaN,...,NaN,NaN,NaN,23,NaN,False,NaN,0.6533337717772364,14,
294b4441caa04a7587a00051e738b19a,FINISHED,1737557013902,1.737557e+12,40984,segment,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,NaN,7,NaN,NaN,NaN,False,NaN,NaN,NaN,
2fcb935e2cb0474dafc839d7069548f9,FINISHED,1737556959605,1.737558e+12,182,satimage,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,6,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,
39be46ef4bcc4230b354dd8e14c8bffb,FINISHED,1737675145917,1.737676e+12,40685,shuttle,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,2,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,


In [29]:
# parameters of each model:
parameters = {
    'RecursiveClustering': ['components_size', 'repetitions', 'kmeans_n_clusters'],
    'KMeans': ['n_clusters'],
    'HDBSCAN': ['min_cluster_size'],
    'DBSCAN': ['eps', 'min_samples'],
    'AffinityPropagation': ['damping'],
    'OPTICS': ['min_samples'],
    'SpectralSubspaceRandomization': ['n_similarities', 'sampling_ratio', 'sc_n_clusters'],
    'WardAgglomerativeClustering': ['n_clusters'],
}

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,mutual_info,n_clusters_,normalized_mutual_info,rand_score,silhouette,v_measure,EXCEPTION,best_child_run_id,parent_run_id,raised_exception
run_uuid,,,,,,,,,,,,,,,,,,,,,
0d3f0312fdfb4b4eb909bcaf25b1703e,FINISHED,1737516206511,1.737518e+12,40984,segment,maximize,adjusted_rand,AffinityPropagation,NaN,NaN,...,0.030963,9.0,0.028502,0.206498,0.371367,0.028502,NaN,0d7f660a5a6145a0a35ff6d18a47cae3,NaN,False
10ac2ec99d13499a8b4d80afbbbe05a3,FINISHED,1737568999692,1.737569e+12,1568,nursery,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,0.412775,14.0,0.218916,0.695642,0.014220,0.218916,NaN,424acca7d3f24ecd86ec6594ea25151b,NaN,False
1577a524c14344f7a86903084f02949e,FINISHED,1737504126260,1.737504e+12,40984,segment,maximize,adjusted_rand,KMeans,NaN,NaN,...,1.179973,14.0,0.564665,0.861749,0.394770,0.564665,NaN,a1f609411ea347128e103bddba078b07,NaN,False
1c48c1907f1447e89c724333a731af26,FINISHED,1737504067203,1.737515e+12,1478,har,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,0.775495,14.0,0.449194,0.772594,-0.018323,0.449194,NaN,de36a2227c364abfa8d85049c6e3d264,NaN,False
1c8422d391974a038aaea93cffd7c20f,FINISHED,1737569000346,1.737569e+12,40984,segment,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,1.117544,14.0,0.541587,0.858334,0.431105,0.541587,NaN,de1a40e3265c4dd29984c438e265d288,NaN,False
27b16ca431d14bf295265e04a20572f0,FINISHED,1737557104228,1.737588e+12,40984,segment,maximize,adjusted_rand,SpectralSubspaceRandomization,NaN,NaN,...,1.461261,17.0,0.649277,0.891340,0.198541,0.649277,NaN,9443a3e882944fc79af9afdc12702716,NaN,False
294b4441caa04a7587a00051e738b19a,FINISHED,1737557013902,1.737557e+12,40984,segment,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,1.130986,41.0,0.517286,0.805215,-0.060018,0.517286,NaN,b85977a952834338891e067174ce5576,NaN,False
2fcb935e2cb0474dafc839d7069548f9,FINISHED,1737556959605,1.737558e+12,182,satimage,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,0.292927,5.0,0.254284,0.437044,0.035195,0.254284,NaN,34778f18c700414da3f4be2f0ddfc780,NaN,False
39be46ef4bcc4230b354dd8e14c8bffb,FINISHED,1737675145917,1.737676e+12,40685,shuttle,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,0.557150,1825.0,0.152159,0.361060,0.045138,0.152159,NaN,5253a2ea464d433fb53b9e10ef74f257,NaN,False


In [27]:
df

Dataset,ecoli,har,iris,nursery,satimage,segment,shuttle
Model,,,,,,,
AffinityPropagation,0.248155,NaN,0.477171,NaN,NaN,0.279673,NaN
DBSCAN,0.345131,0.302146,0.558371,0.000000,0.296792,0.250797,0.685962
HDBSCAN,0.397572,0.286958,0.563751,0.014181,0.308123,0.390071,0.001018
KMeans,0.718701,0.438310,0.592333,0.150396,0.566324,0.512024,0.608404
OPTICS,0.313694,0.000694,0.396136,0.000000,0.022515,0.097270,0.016784
RecursiveClustering,0.758326,0.491249,0.631048,0.440254,0.583496,0.539809,0.651919
SpectralSubspaceRandomization,NaN,NaN,NaN,NaN,NaN,0.481409,NaN
WardAgglomerativeClustering,0.734778,0.510766,0.615323,0.253798,0.484501,0.446081,0.478068


In [52]:
df = df_runs_raw_parents.copy()
df = df.loc[df['model_nickname'] == 'RecursiveClustering']
df = df.loc[df['metric'] == 'adjusted_rand']
df = df.loc[(df['model_params/normalization'] == '1') | (df['standardize'] == 'True')]
df = df[['model_nickname', 'dataset_name', 'dataset_id', 'model_params/normalization', 'standardize', 'best_adjusted_rand']]

In [56]:
df = df.loc[df.duplicated(subset='dataset_id', keep=False)]
df = df.sort_values(['dataset_id', 'best_adjusted_rand'])

In [57]:
df

key,model_nickname,dataset_name,dataset_id,model_params/normalization,standardize,best_adjusted_rand
run_uuid,,,,,,
864ffa472636448f8c818a03dac0e102,RecursiveClustering,mfeat-factors,12,1,False,0.512992
48bf194dc53e40e39258a1b6ad6ef4fa,RecursiveClustering,mfeat-factors,12,NaN,True,0.541314
3636b6d774d3490eb5564cc679270785,RecursiveClustering,mfeat-morphological,18,1,False,0.562850
37fc83ca94d644208788c721ad681af7,RecursiveClustering,mfeat-morphological,18,NaN,True,0.573092
f0e8851a821345bdab9b6527e0d3cc36,RecursiveClustering,satimage,182,1,False,0.556990
50f3fd8149324f6da7314c1d27cc06cf,RecursiveClustering,satimage,182,NaN,True,0.583496
4b37b3fcbfb9414ab3b59a49e1696de2,RecursiveClustering,page-blocks,30,NaN,True,0.572026
eec8aab4384b4a6f8e0aea1e2af25d2e,RecursiveClustering,page-blocks,30,1,False,0.603016
d1bb6eb83a6943ea8be2497c99fba63c,RecursiveClustering,pendigits,32,1,False,0.615024


In [75]:
df = df_metrics.copy()
df = df.loc[df.index.get_level_values('run_uuid') == 'eec8aab4384b4a6f8e0aea1e2af25d2e']

In [80]:
x_metric = 'silhouette'
df = df.sort_values(x_metric)
px.scatter(df, x=x_metric, y='adjusted_rand')

In [109]:
df = df_common.copy()
df = df.loc[df['n_samples'] == 1000]
df = df.loc[df['n_features'] == 1000]
df = df.loc[df['pct_random'] == 0.5]
df = df.loc[df['class_sep'] == 100.0]
df = df.sort_values('model_nickname')
fig = px.box(df, x='model_nickname', y='best_adjusted_rand', color='model_nickname')
fig.show()

In [169]:
df = df_common.copy()
df = df.loc[df['n_samples'] == 1000]
df = df.loc[df['n_features'] == 1000]
# df = df.loc[df['pct_random'] == 0.7]
df = df.loc[df['class_sep'] == 100.0]
df = df.loc[df['model_nickname'].isin(['RecursiveClustering', 'KMeans', 'HDBSCAN', 'DBSCAN', 'AffinityPropagation', 'OPTICS', 'SpectralSubspaceRandomization', 'WardAgglomerativeClustering'])]
df = df.sort_values('model_nickname')
df = df.rename(columns={'fit_model_return_elapsed_time': 'Time (s)', 'max_memory_used': 'Memory (MB)', 'n_samples': 'Number of samples', 'n_features': 'Number of features', 'model_nickname': 'Model', 'best_adjusted_rand': 'Adjusted Rand', 'best_n_clusters_': 'Number of Clusters Found', 'n_classes': 'True Number of Clusters', 'class_sep': 'Class Separation', 'pct_random': '% Random Features', 'seed_unified': 'Seed'})
df = df[['Model', 'Adjusted Rand', 'Time (s)', 'Memory (MB)', 'Number of samples', 'Number of features', 'Class Separation', '% Random Features', 'Seed', 'Number of Clusters Found', 'True Number of Clusters']]
df = df.groupby('Model').agg({'Adjusted Rand': ['mean', 'std'], 'Number of Clusters Found': ['median', ('Q25', lambda x: x.quantile(0.25)), ('Q75', lambda x: x.quantile(0.75))], 'Time (s)': ['mean', 'std'], 'Memory (MB)': ['mean', 'std'], 'Number of samples': ['mean', 'nunique'], 'Number of features': ['mean', 'nunique'], 'Class Separation': ['mean', 'nunique'], '% Random Features': ['mean', 'nunique'], 'True Number of Clusters': ['mean', 'nunique'], 'Seed': ['mean', 'nunique']})
df.to_csv('simulated_data_easy.csv')
df

key                           Adjusted Rand      Number of Clusters Found  \
                                       mean  std                   median   
Model                                                                       
AffinityPropagation                     1.0  0.0                      5.0   
DBSCAN                                  0.0  0.0                      1.0   
HDBSCAN                                 1.0  0.0                      5.0   
KMeans                                  1.0  0.0                      5.0   
OPTICS                                  1.0  0.0                      5.0   
RecursiveClustering                     1.0  0.0                      5.0   
SpectralSubspaceRandomization           1.0  0.0                      5.0   
WardAgglomerativeClustering             1.0  0.0                      5.0   

key                                         Time (s)             Memory (MB)  \
                               Q25  Q75         mean         std        mean   
Model                                                                          
AffinityPropagation            5.0  5.0   537.226421   39.111352   2059.4429   
DBSCAN                         1.0  1.0   495.724737   24.503844   2034.5569   
HDBSCAN                        5.0  5.0   156.379071   16.678345   1951.1594   
KMeans                         5.0  5.0   455.211977   39.756185   2024.6738   
OPTICS                         5.0  5.0   186.481377   12.323363   2067.3815   
RecursiveClustering            5.0  5.0  2264.785293  217.413369   2080.0290   
SpectralSubspaceRandomization  5.0  5.0  3496.919557  733.693875   1108.6247   
WardAgglomerativeClustering    5.0  5.0   267.337123   30.720733   2059.1330   

key                                       Number of samples  ...  \
                                      std              mean  ...   
Model                                                        ...   
AffinityPropagation              1.779558            1000.0  ...   
DBSCAN                           1.282687            1000.0  ...   
HDBSCAN                        228.016285            1000.0  ...   
KMeans                         151.542389            1000.0  ...   
OPTICS                           0.839997            1000.0  ...   
RecursiveClustering              2.354198            1000.0  ...   
SpectralSubspaceRandomization   27.026635            1000.0  ...   
WardAgglomerativeClustering      1.695819            1000.0  ...   

key                           Number of features         Class Separation  \
                                            mean nunique             mean   
Model                                                                       
AffinityPropagation                       1000.0       1            100.0   
DBSCAN                                    1000.0       1            100.0   
HDBSCAN                                   1000.0       1            100.0   
KMeans                                    1000.0       1            100.0   
OPTICS                                    1000.0       1            100.0   
RecursiveClustering                       1000.0       1            100.0   
SpectralSubspaceRandomization             1000.0       1            100.0   
WardAgglomerativeClustering               1000.0       1            100.0   

key                                   % Random Features          \
                              nunique              mean nunique   
Model                                                             
AffinityPropagation                 1              0.35       4   
DBSCAN                              1              0.35       4   
HDBSCAN                             1              0.35       4   
KMeans                              1              0.35       4   
OPTICS                              1              0.35       4   
RecursiveClustering                 1              0.35       4   
SpectralSubspaceRandomization       1              0.35       4   
Ward

In [108]:
df = df_common.copy()
df = df.loc[df['n_samples'] == 1000]
df = df.loc[df['n_features'] == 1000]
df = df.loc[df['pct_random'] == 0.5]
df = df.loc[df['class_sep'] == 10.0]
df = df.loc[df['model_nickname'].isin(['RecursiveClustering', 'KMeans', 'HDBSCAN', 'DBSCAN', 'AffinityPropagation', 'OPTICS', 'SpectralSubspaceRandomization', 'WardAgglomerativeClustering'])]
df = df.sort_values('model_nickname')
df = df.rename(columns={'fit_model_return_elapsed_time': 'Time (s)', 'max_memory_used': 'Memory (MB)', 'n_samples': 'Number of samples', 'n_features': 'Number of features', 'model_nickname': 'Model', 'best_adjusted_rand': 'Adjusted Rand', 'best_n_clusters_': 'Number of Clusters Found', 'n_classes': 'True Number of Clusters', 'class_sep': 'Class Separation', 'pct_random': '% Random Features', 'seed_unified': 'Seed'})
df = df[['Model', 'Adjusted Rand', 'Time (s)', 'Memory (MB)', 'Number of samples', 'Number of features', 'Class Separation', '% Random Features', 'Seed', 'Number of Clusters Found', 'True Number of Clusters']]
df = df.groupby('Model').agg({'Adjusted Rand': ['mean', 'std'], 'Number of Clusters Found': ['median', ('Q25', lambda x: x.quantile(0.25)), ('Q75', lambda x: x.quantile(0.75))], 'Time (s)': ['mean', 'std'], 'Memory (MB)': ['mean', 'std'], 'Number of samples': ['mean', 'nunique'], 'Number of features': ['mean', 'nunique'], 'Class Separation': ['mean', 'nunique'], '% Random Features': ['mean', 'nunique'], 'True Number of Clusters': ['mean', 'nunique'], 'Seed': ['mean', 'nunique']})
df.to_csv('simulated_data_hard.csv')
df

key                           Adjusted Rand            \
                                       mean       std   
Model                                                   
AffinityPropagation                0.189093  0.017298   
DBSCAN                             0.000000  0.000000   
HDBSCAN                            0.993713  0.003036   
KMeans                             0.200988  0.060978   
OPTICS                             0.399327  0.344323   
RecursiveClustering                0.433655  0.230195   
SpectralSubspaceRandomization      0.915287  0.202475   
WardAgglomerativeClustering        0.494884  0.296377   

key                           Number of Clusters Found              \
                                                median  Q25    Q75   
Model                                                                
AffinityPropagation                               21.5  2.0  42.00   
DBSCAN                                             1.0  1.0   1.00   
HDBSCAN                                            6.0  6.0   6.00   
KMeans                                             2.0  2.0   2.00   
OPTICS                                             2.5  2.0   4.50   
RecursiveClustering                                4.0  3.0   4.75   
SpectralSubspaceRandomization                      5.0  5.0   5.00   
WardAgglomerativeClustering                        2.0  2.0   4.25   

key                               Time (s)             Memory (MB)  \
                                      mean         std        mean   
Model                                                                
AffinityPropagation             529.730182   10.693666   2059.4900   
DBSCAN                          478.425069    8.144741   2034.9944   
HDBSCAN                         157.498777    5.076115   2008.3772   
KMeans                          421.863799   23.231615   2065.8152   
OPTICS                          193.670200    4.669891   2067.3468   
RecursiveClustering            2390.756949  168.434082   2078.1840   
SpectralSubspaceRandomization  3650.157271  473.980725   1103.1100   
WardAgglomerativeClustering     250.402186   13.845500   2058.1964   

key                                       Number of samples  ...  \
                                      std              mean  ...   
Model                                                        ...   
AffinityPropagation              2.024502            1000.0  ...   
DBSCAN                           1.190787            1000.0  ...   
HDBSCAN                        178.319054            1000.0  ...   
KMeans                           4.504178            1000.0  ...   
OPTICS                           1.081793            1000.0  ...   
RecursiveClustering              2.627123            1000.0  ...   
SpectralSubspaceRandomization    4.184572            1000.0  ...   
WardAgglomerativeClustering      2.105722            1000.0  ...   

key                           Number of features         Class Separation  \
                                            mean nunique             mean   
Model                                                                       
AffinityPropagation                       1000.0       1             10.0   
DBSCAN                                    1000.0       1             10.0   
HDBSCAN                                   1000.0       1             10.0   
KMeans                                    1000.0       1             10.0   
OPTICS                                    1000.0       1             10.0   
RecursiveClustering                       1000.0       1             10.0   
SpectralSubspaceRandomization             1000.0       1             10.0   
WardAgglomerativeClustering               1000.0       1             10.0   

key                                   % Random Features          \
                              nunique              mean nunique   
Model                                                             
AffinityPropagation             

# With filling values

In [132]:
non_duplicate_columns = [
    'model_nickname',
    'n_samples',
    'n_features',
    'pct_random',
    'seed_unified',
    'class_sep',
    'n_random',
    'n_informative',
    'n_classes'
]
df_runs_parents_fill = df_runs_raw_parents.dropna(axis=0, how='all', subset=['best_adjusted_rand']).copy()
df_runs_parents_fill = df_runs_parents_fill.loc[(~df_runs_parents_fill.duplicated(non_duplicate_columns))]

In [133]:
# for the moment, I did not manage to run HDBSCAN and SpectralSubspaceRandomization with samples >= 10000, so we will fill these values for the moment as "no run" and we will see later
status = 'FAILED'
no_run_time = np.inf
no_run_memory = np.inf
no_run_metric = np.nan
raised_exception = True
EXCEPTION = 'NoRun'
start_time = time.time() * 1000
end_time = time.time() * 1000

In [134]:
model_nickname = 'HDBSCAN'
n_samples = ['10000']
n_features = ['100', '1000', '10000']
pct_random = ['0.0', '0.2', '0.5', '0.7']
seed_unified = [f'{i}' for i in range(10)]
class_sep = ['10.0', '100.0', '50.0']
n_classes = 5
combinations = list(product(n_samples, n_features, pct_random, seed_unified, class_sep))
df_cat = []
indexes = []
for n_sample, n_feature, pct_r, seed, class_s in combinations:
    df = {}
    index = f'{model_nickname}_{n_sample}_{n_feature}'
    df['status'] = status
    df['start_time'] = start_time
    df['end_time'] = end_time
    df['model_nickname'] = model_nickname
    df['n_classes'] = n_classes
    df['n_features'] = n_feature
    df['n_samples'] = n_sample
    df['pct_random'] = pct_r
    df['seed_unified'] = seed
    df['class_sep'] = class_s
    df['fit_model_return_elapsed_time'] = no_run_time
    df['max_memory_used'] = no_run_memory
    df['best_adjusted_rand'] = no_run_metric
    df['best_n_clusters_'] = no_run_metric
    df['EXCEPTION'] = EXCEPTION
    df['raised_exception'] = raised_exception
    df_cat.append(df)
    indexes.append(index)
df_no_run = pd.DataFrame(df_cat, index=indexes)
df_runs_parents_fill = pd.concat([df_runs_parents_fill, df_no_run])

In [135]:
model_nickname = 'SpectralSubspaceRandomization'
n_samples = ['10000']
n_features = ['100', '1000', '10000']
pct_random = ['0.0', '0.2', '0.5', '0.7']
seed_unified = [f'{i}' for i in range(10)]
class_sep = ['10.0', '100.0', '50.0']
n_classes = 5
combinations = list(product(n_samples, n_features, pct_random, seed_unified, class_sep))
df_cat = []
indexes = []
for n_sample, n_feature, pct_r, seed, class_s in combinations:
    df = {}
    index = f'{model_nickname}_{n_sample}_{n_feature}'
    df['status'] = status
    df['start_time'] = start_time
    df['end_time'] = end_time
    df['model_nickname'] = model_nickname
    df['n_classes'] = n_classes
    df['n_features'] = n_feature
    df['n_samples'] = n_sample
    df['pct_random'] = pct_r
    df['seed_unified'] = seed
    df['class_sep'] = class_s
    df['fit_model_return_elapsed_time'] = no_run_time
    df['max_memory_used'] = no_run_memory
    df['best_adjusted_rand'] = no_run_metric
    df['best_n_clusters_'] = no_run_metric
    df['EXCEPTION'] = EXCEPTION
    df['raised_exception'] = raised_exception
    df_cat.append(df)
    indexes.append(index)
df_no_run = pd.DataFrame(df_cat, index=indexes)
df_runs_parents_fill = pd.concat([df_runs_parents_fill, df_no_run])

In [136]:
# ensure no duplicates
df_runs_parents_fill = df_runs_parents_fill.loc[(~df_runs_parents_fill.duplicated(non_duplicate_columns))]

# Missing

In [137]:
df_runs_parents_fill['model_nickname'].unique().tolist()

['RecursiveClustering',
 'AverageAgglomerativeClustering',
 'KMeansProj',
 'HDBSCAN',
 'OPTICS',
 'SpectralSubspaceRandomization',
 'Proclus',
 'DBSCAN',
 'WardAgglomerativeClustering',
 'KMeans',
 'SingleAgglomerativeClustering',
 'SpectralClustering',
 'CompleteAgglomerativeClustering',
 'IRFLLRR',
 'Clique',
 'AffinityPropagation']

In [138]:
non_duplicate_columns = [
    'model_nickname',
    'n_samples',
    'n_features',
    'pct_random',
    'seed_unified',
    'class_sep',
]

In [139]:
df_runs_parents_fill

,status,start_time,end_time,class_sep,model_nickname,n_classes,n_features,n_informative,n_random,n_samples,...,max_memory_used,mutual_info,n_clusters_,normalized_mutual_info,rand_score,silhouette,EXCEPTION,best_child_run_id,parent_run_id,raised_exception
000b291a142341289479933ddb447488,FINISHED,1.736872e+12,1.736877e+12,50.0,RecursiveClustering,5,10000,2,16,1000,...,1156.568,1.609438,577.0,0.425927,0.806024,-0.104928,NaN,e62023cb27d941dd8b9683eaef947ccc,NaN,False
000f29a52641434a9a03ec3951ba12ba,FINISHED,1.736890e+12,1.736911e+12,50.0,AverageAgglomerativeClustering,5,10000,2,16,10000,...,1941.040,1.608578,8.0,0.997419,0.999561,0.283838,NaN,8e0a1045c9474ca5925d553aa6030f20,NaN,False
0015c6675ce048ec99ded8242b2df23e,FINISHED,1.734337e+12,1.734515e+12,100.0,KMeansProj,5,100,2,16,1000,...,473.880,1.079833,12.0,0.539120,0.835267,-0.002984,NaN,d4a718cc5d92496ba63ebf72357e77db,NaN,False
0018167e4ff1486a8a909f25fdaa5d76,FINISHED,1.734289e+12,1.734364e+12,100.0,RecursiveClustering,5,100,2,16,1000,...,2077.516,1.609438,5.0,1.000000,1.000000,0.936638,NaN,0fc46cf1ca434f86841b195a2b92e37d,NaN,False
00187fd8f9df4282ba536e067e9b6881,FINISHED,1.736687e+12,1.736691e+12,100.0,HDBSCAN,5,100,2,16,10000,...,1212.672,1.609438,5.0,1.000000,1.000000,0.939268,NaN,c25dc85141e84c21b389b9e9cfced842,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SpectralSubspaceRandomization_10000_10000,FAILED,1.736947e+12,1.736947e+12,100.0,SpectralSubspaceRandomization,5,10000,NaN,NaN,10000,...,inf,NaN,NaN,NaN,NaN,NaN,NoRun,NaN,NaN,True
SpectralSubspaceRandomization_10000_10000,FAILED,1.736947e+12,1.736947e+12,50.0,SpectralSubspaceRandomization,5,10000,NaN,NaN,10000,...,inf,NaN,NaN,NaN,NaN,NaN,NoRun,NaN,NaN,True
SpectralSubspaceRandomization_10000_10000,FAILED,1.736947e+12,1.736947e+12,10.0,SpectralSubspaceRandomization,5,10000,NaN,NaN,10000,...,inf,NaN,NaN,NaN,NaN,NaN,NoRun,NaN,NaN,True
SpectralSubspaceRandomization_10000_10000,FAILED,1.736947e+12,1.736947e+12,100.0,SpectralSubspaceRandomization,5,10000,NaN,NaN,10000,...,inf,NaN,NaN,NaN,NaN,NaN,NoRun,NaN,NaN,True


In [144]:
n_samples = ['100', '1000', '10000']
n_features = ['100', '1000', '10000']
pct_random = ['0.0', '0.2', '0.5', '0.7']
seed_unified = [f'{i}' for i in range(1)]
class_sep = ['10.0', '100.0']
model_nickname = [
    # 'KMeansProj',
    # 'IRFLLRR',
    # 'Clique',
    'HDBSCAN',
    'SpectralSubspaceRandomization',
    'SingleAgglomerativeClustering',
    'SpectralClustering',
    'RecursiveClustering',
    'OPTICS',
    'Proclus',
    'WardAgglomerativeClustering',
    'KMeans',
    'DBSCAN',
    'CompleteAgglomerativeClustering',
    'AverageAgglomerativeClustering',
    'AffinityPropagation'
]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, n_samples, n_features, pct_random, seed_unified, class_sep]
df_missing = get_missing_entries(df_runs_parents_fill, columns_names, should_contain_values)

In [145]:
df_missing

,model_nickname,n_samples,n_features,pct_random,seed_unified,class_sep
0,SingleAgglomerativeClustering,10000,10000,0.0,0,10.0
1,SingleAgglomerativeClustering,10000,10000,0.0,0,100.0
2,SingleAgglomerativeClustering,10000,10000,0.2,0,10.0
3,SingleAgglomerativeClustering,10000,10000,0.2,0,100.0
4,SingleAgglomerativeClustering,10000,10000,0.5,0,10.0
5,SingleAgglomerativeClustering,10000,10000,0.5,0,100.0
6,SingleAgglomerativeClustering,10000,10000,0.7,0,10.0
7,SingleAgglomerativeClustering,10000,10000,0.7,0,100.0
8,SpectralClustering,10000,100,0.5,0,10.0
9,SpectralClustering,10000,100,0.5,0,100.0


# Get common combinations

In [146]:
model_nickname = [
    # 'KMeansProj',
    # 'IRFLLRR',
    # 'Clique',
    'HDBSCAN',
    'SpectralSubspaceRandomization',
    # 'SingleAgglomerativeClustering',
    # 'SpectralClustering',
    'RecursiveClustering',
    'OPTICS',
    'Proclus',
    'WardAgglomerativeClustering',
    'KMeans',
    'DBSCAN',
    'CompleteAgglomerativeClustering',
    'AverageAgglomerativeClustering',
    'AffinityPropagation'
]
df = df_runs_parents_fill.copy()
df = df.loc[df['model_nickname'].isin(model_nickname)]
column = 'model_nickname'
combination_columns = [
    'n_samples',
    'n_features',
    'pct_random',
    'seed_unified',
    'class_sep',
]
common_combinations = get_common_combinations(df, column, combination_columns)

In [147]:
df_common_fill = get_df_with_combinations(df, combination_columns, common_combinations)

In [148]:
df_common_fill['n_samples'] = df_common_fill['n_samples'].astype(int)
df_common_fill['n_features'] = df_common_fill['n_features'].astype(int)
df_common_fill['pct_random'] = df_common_fill['pct_random'].astype(float)
df_common_fill['class_sep'] = df_common_fill['class_sep'].astype(float)
df_common_fill['seed_unified'] = df_common_fill['seed_unified'].astype(int)
df_common_fill['n_classes'] = df_common_fill['n_classes'].astype(int)

## Plot

In [154]:
df

,status,start_time,end_time,class_sep,model_nickname,n_classes,n_features,n_informative,n_random,n_samples,...,max_memory_used,mutual_info,n_clusters_,normalized_mutual_info,rand_score,silhouette,EXCEPTION,best_child_run_id,parent_run_id,raised_exception
9b5af53d2adb4e66b6c299cc9dd03f84,FINISHED,1.736864e+12,1.736871e+12,100.0,AffinityPropagation,5,1000,2,16,10000,...,4457.724,1.609438,18.0,0.719116,0.858192,0.084554,NaN,8b29db04c53947fb8a6298af4012597f,NaN,False
6e13b2c209784d15ad55255aff27e0fd,FINISHED,1.736864e+12,1.736868e+12,100.0,AverageAgglomerativeClustering,5,1000,2,16,10000,...,1231.704,1.609438,14.0,0.963381,0.989579,0.261144,NaN,2a961b3635994d84a2494e22d57f7e76,NaN,False
6fc65459261041fda9074a00019fee06,FINISHED,1.736688e+12,1.736808e+12,100.0,CompleteAgglomerativeClustering,5,1000,2,16,10000,...,3575.068,1.609438,14.0,0.779245,0.889110,0.079613,NaN,61cc5cae6000412bbb61cdaf2bf7fec7,NaN,False
37261cab80634ea7b30a76a55b63d541,FINISHED,1.736686e+12,1.736694e+12,100.0,DBSCAN,5,1000,2,16,10000,...,1918.316,0.000000,1.0,0.000000,0.199920,-1.000000,NaN,f93e17338fdb4361a1ab9ca7acb61d9f,NaN,False
a88089c38f8548eab4a4f5a687b268c5,FINISHED,1.736687e+12,1.736701e+12,100.0,HDBSCAN,5,1000,2,16,10000,...,1301.976,1.609438,5.0,1.000000,1.000000,0.839192,NaN,d07a9c27fc094cd0b10e2291625f6a04,NaN,False
HDBSCAN_10000_1000,FAILED,1.736947e+12,1.736947e+12,100.0,HDBSCAN,5,1000,NaN,NaN,10000,...,inf,NaN,NaN,NaN,NaN,NaN,NoRun,NaN,NaN,True
b5e3972f272d47199e54bb7d9a1ae019,FINISHED,1.736866e+12,1.736866e+12,100.0,KMeans,5,1000,2,16,10000,...,1190.328,1.609438,14.0,0.760001,0.873464,0.110790,NaN,339825b5dad64b748b8ba9fec4b5dc3d,NaN,False
4d07fcc980ec4f939220cb15718a0d46,FINISHED,1.736707e+12,1.736746e+12,100.0,OPTICS,5,1000,2,16,10000,...,1287.588,1.609438,5.0,1.000000,1.000000,0.839192,NaN,482d10afeb4e44a2b3a78a35966e8f00,NaN,False
ffaaa267a0634c8786d45d623e7a5a6a,FINISHED,1.736708e+12,1.736767e+12,100.0,Proclus,5,1000,2,16,10000,...,1697.296,0.000755,4.0,0.000523,0.617884,-0.007934,NaN,489a115c31194f1f878d8961d7888b3a,NaN,False
9bc581d07a254b95a7c77a44afee74a2,FINISHED,1.736791e+12,1.736824e+12,100.0,RecursiveClustering,5,1000,2,16,10000,...,2186.084,1.609438,7.0,0.999242,0.999880,0.623656,NaN,f161eb41e12e4109a0a29037aa0e7659,NaN,False


In [192]:
df = df_common_fill.copy()
df = df.loc[df['n_samples'] == 10000]
df = df.loc[df['n_features'] == 10000]
# df = df.loc[df['pct_random'] == 0.5]
df = df.loc[df['class_sep'] == 50.0]
df = df.sort_values('model_nickname')
fig = px.box(df, x='model_nickname', y='best_adjusted_rand', color='model_nickname')
fig.show()

In [160]:
df = df_common_fill.copy()
df = df.loc[df['n_samples'] == 10000]
df = df.loc[df['n_features'] == 10000]
# df = df.loc[df['pct_random'] == 0.5]
df = df.loc[df['class_sep'] == 100.0]
df = df.loc[df['model_nickname'].isin(['RecursiveClustering', 'KMeans', 'HDBSCAN', 'DBSCAN', 'AffinityPropagation', 'OPTICS', 'SpectralSubspaceRandomization', 'WardAgglomerativeClustering'])]
df = df.sort_values('model_nickname')
df = df.rename(columns={'fit_model_return_elapsed_time': 'Time (s)', 'max_memory_used': 'Memory (MB)', 'n_samples': 'Number of samples', 'n_features': 'Number of features', 'model_nickname': 'Model', 'best_adjusted_rand': 'Adjusted Rand', 'best_n_clusters_': 'Number of Clusters Found', 'n_classes': 'True Number of Clusters', 'class_sep': 'Class Separation', 'pct_random': '% Random Features', 'seed_unified': 'Seed'})
df = df[['Model', 'Adjusted Rand', 'Time (s)', 'Memory (MB)', 'Number of samples', 'Number of features', 'Class Separation', '% Random Features', 'Seed', 'Number of Clusters Found', 'True Number of Clusters']]
df = df.groupby('Model').agg({'Adjusted Rand': ['mean', 'std'], 'Number of Clusters Found': ['median', ('Q25', lambda x: x.quantile(0.25)), ('Q75', lambda x: x.quantile(0.75))], 'Time (s)': ['mean', 'std'], 'Memory (MB)': ['mean', 'std'], 'Number of samples': ['mean', 'nunique'], 'Number of features': ['mean', 'nunique'], 'Class Separation': ['mean', 'nunique'], '% Random Features': ['mean', 'nunique'], 'True Number of Clusters': ['mean', 'nunique'], 'Seed': ['mean', 'nunique']})
df.to_csv('simulated_data_easy_more_samples.csv')
df

Adjusted Rand            \
                                       mean       std   
Model                                                   
AffinityPropagation                0.160395  0.040795   
DBSCAN                             0.000000  0.000000   
HDBSCAN                                 NaN       NaN   
KMeans                             1.000000  0.000000   
OPTICS                             1.000000  0.000000   
RecursiveClustering                1.000000  0.000000   
SpectralSubspaceRandomization           NaN       NaN   
WardAgglomerativeClustering        0.782541  0.000000   

                              Number of Clusters Found               \
                                                median    Q25   Q75   
Model                                                                 
AffinityPropagation                               57.0  47.75  68.5   
DBSCAN                                             1.0   1.00   1.0   
HDBSCAN                                            NaN    NaN   NaN   
KMeans                                             5.0   5.00   5.0   
OPTICS                                             5.0   5.00   5.0   
RecursiveClustering                                5.0   5.00   5.0   
SpectralSubspaceRandomization                      NaN    NaN   NaN   
WardAgglomerativeClustering                        4.0   4.00   4.0   

                                  Time (s)             Memory (MB)  \
                                      mean         std        mean   
Model                                                                
AffinityPropagation            6950.536954  455.081972    4258.068   
DBSCAN                         1035.232506   75.605361    3573.426   
HDBSCAN                                inf         NaN         inf   
KMeans                          749.472146  169.602759    1938.543   
OPTICS                         9990.183728  510.976754    1981.357   
RecursiveClustering            3492.044976  772.109737    2571.013   
SpectralSubspaceRandomization          inf         NaN         inf   
WardAgglomerativeClustering    7430.754580  155.178050    3571.057   

                                           Number of samples  ...  \
                                       std              mean  ...   
Model                                                         ...   
AffinityPropagation              15.686925           10000.0  ...   
DBSCAN                            1.666874           10000.0  ...   
HDBSCAN                                NaN           10000.0  ...   
KMeans                          863.966482           10000.0  ...   
OPTICS                            6.291208           10000.0  ...   
RecursiveClustering            1067.581430           10000.0  ...   
SpectralSubspaceRandomization          NaN           10000.0  ...   
WardAgglomerativeClustering       2.912283           10000.0  ...   

                              Number of features         Class Separation  \
                                            mean nunique             mean   
Model                                                                       
AffinityPropagation                      10000.0       1            100.0   
DBSCAN                                   10000.0       1            100.0   
HDBSCAN                                  10000.0       1            100.0   
KMeans                                   10000.0       1            100.0   
OPTICS                                   10000.0       1            100.0   
RecursiveClustering                      10000.0       1            100.0   
SpectralSubspaceRandomization            10000.0       1            100.0   
WardAgglomerativeClustering              10000.0       1            100.0   

                                      % Random Features          \
                              nunique              mean nunique   
Model                                                             
AffinityPropagation                 1   

In [177]:
df = df_common_fill.copy()
df = df.loc[df['n_samples'] == 10000]
df = df.loc[df['n_features'] == 1000]
df = df.loc[df['pct_random'] == 0.5]
df = df.loc[df['class_sep'] == 10.0]
df = df.loc[df['model_nickname'].isin(['RecursiveClustering', 'KMeans', 'HDBSCAN', 'DBSCAN', 'AffinityPropagation', 'OPTICS', 'SpectralSubspaceRandomization', 'WardAgglomerativeClustering'])]
df = df.sort_values('model_nickname')
df = df.rename(columns={'fit_model_return_elapsed_time': 'Time (s)', 'max_memory_used': 'Memory (MB)', 'n_samples': 'Number of samples', 'n_features': 'Number of features', 'model_nickname': 'Model', 'best_adjusted_rand': 'Adjusted Rand', 'best_n_clusters_': 'Number of Clusters Found', 'n_classes': 'True Number of Clusters', 'class_sep': 'Class Separation', 'pct_random': '% Random Features', 'seed_unified': 'Seed'})
df = df[['Model', 'Adjusted Rand', 'Time (s)', 'Memory (MB)', 'Number of samples', 'Number of features', 'Class Separation', '% Random Features', 'Seed', 'Number of Clusters Found', 'True Number of Clusters']]
df = df.groupby('Model').agg({'Adjusted Rand': ['mean', 'std'], 'Number of Clusters Found': ['median', ('Q25', lambda x: x.quantile(0.25)), ('Q75', lambda x: x.quantile(0.75))], 'Time (s)': ['mean', 'std'], 'Memory (MB)': ['mean', 'std'], 'Number of samples': ['mean', 'nunique'], 'Number of features': ['mean', 'nunique'], 'Class Separation': ['mean', 'nunique'], '% Random Features': ['mean', 'nunique'], 'True Number of Clusters': ['mean', 'nunique'], 'Seed': ['mean', 'nunique']})
df.to_csv('simulated_data_hard_more_samples.csv')
df

Adjusted Rand     Number of Clusters Found  \
                                       mean std                   median   
Model                                                                      
AffinityPropagation            4.081970e-01 NaN                      5.0   
DBSCAN                         0.000000e+00 NaN                      1.0   
HDBSCAN                        9.993746e-01 NaN                      6.0   
KMeans                         1.383373e-01 NaN                      2.0   
OPTICS                         9.270328e-07 NaN                     18.0   
RecursiveClustering            4.094620e-01 NaN                      3.0   
SpectralSubspaceRandomization           NaN NaN                      NaN   
WardAgglomerativeClustering    9.457308e-01 NaN                      5.0   

                                              Time (s)     Memory (MB)      \
                                Q25   Q75         mean std        mean std   
Model                                                                        
AffinityPropagation             5.0   5.0  7348.131929 NaN    3590.168 NaN   
DBSCAN                          1.0   1.0   406.390341 NaN    2165.288 NaN   
HDBSCAN                         6.0   6.0          inf NaN         inf NaN   
KMeans                          2.0   2.0   269.914718 NaN    1187.536 NaN   
OPTICS                         18.0  18.0  4420.699471 NaN    1287.016 NaN   
RecursiveClustering             3.0   3.0  7003.752386 NaN    2194.076 NaN   
SpectralSubspaceRandomization   NaN   NaN          inf NaN         inf NaN   
WardAgglomerativeClustering     5.0   5.0  1533.686058 NaN    1308.080 NaN   

                              Number of samples  ... Number of features  \
                                           mean  ...               mean   
Model                                            ...                      
AffinityPropagation                     10000.0  ...             1000.0   
DBSCAN                                  10000.0  ...             1000.0   
HDBSCAN                                 10000.0  ...             1000.0   
KMeans                                  10000.0  ...             1000.0   
OPTICS                                  10000.0  ...             1000.0   
RecursiveClustering                     10000.0  ...             1000.0   
SpectralSubspaceRandomization           10000.0  ...             1000.0   
WardAgglomerativeClustering             10000.0  ...             1000.0   

                                      Class Separation          \
                              nunique             mean nunique   
Model                                                            
AffinityPropagation                 1             10.0       1   
DBSCAN                              1             10.0       1   
HDBSCAN                             1             10.0       1   
KMeans                              1             10.0       1   
OPTICS                              1             10.0       1   
RecursiveClustering                 1             10.0       1   
SpectralSubspaceRandomization       1             10.0       1   
WardAgglomerativeClustering         1             10.0       1   

                              % Random Features          \
                                           mean nunique   
Model                                                     
AffinityPropagation                         0.5       1   
DBSCAN                                      0.5       1   
HDBSCAN                                     0.5       1   
KMeans                                      0.5       1   
OPTICS                                      0.5       1   
RecursiveClustering                         0.5       1   
SpectralSubspaceRandomization               0.5       1   
WardAgglomerativeClustering                 0.5       1   

                              True Number of Clusters         Seed          
                                                 mean 

In [195]:
df = df_common_fill.copy()
n_samples = 100
df = df.loc[df['n_samples'] == n_samples]
df = df.loc[df['n_features'] == 10000]
# df = df.loc[df['pct_random'] == 0.5]
df = df.loc[df['class_sep'] == 50.0]
df = df.loc[df['model_nickname'].isin(['RecursiveClustering', 'KMeans', 'HDBSCAN', 'DBSCAN', 'AffinityPropagation', 'OPTICS', 'SpectralSubspaceRandomization', 'WardAgglomerativeClustering'])]
df = df.sort_values('model_nickname')
df = df.rename(columns={'fit_model_return_elapsed_time': 'Time (s)', 'max_memory_used': 'Memory (MB)', 'n_samples': 'Number of samples', 'n_features': 'Number of features', 'model_nickname': 'Model', 'best_adjusted_rand': 'Adjusted Rand', 'best_n_clusters_': 'Number of Clusters Found', 'n_classes': 'True Number of Clusters', 'class_sep': 'Class Separation', 'pct_random': '% Random Features', 'seed_unified': 'Seed'})
df = df[['Model', 'Adjusted Rand', 'Time (s)', 'Memory (MB)', 'Number of samples', 'Number of features', 'Class Separation', '% Random Features', 'Seed', 'Number of Clusters Found', 'True Number of Clusters']]
df = df.groupby('Model').agg({'Adjusted Rand': ['mean', 'std'], 'Number of Clusters Found': ['median', ('Q25', lambda x: x.quantile(0.25)), ('Q75', lambda x: x.quantile(0.75))], 'Time (s)': ['mean', 'std'], 'Memory (MB)': ['mean', 'std'], 'Number of samples': ['mean', 'nunique'], 'Number of features': ['mean', 'nunique'], 'Class Separation': ['mean', 'nunique'], '% Random Features': ['mean', 'nunique'], 'True Number of Clusters': ['mean', 'nunique'], 'Seed': ['mean', 'nunique']})
df.to_csv(f'simulated_data_medium_{n_samples}samples.csv')
df

Adjusted Rand            \
                                       mean       std   
Model                                                   
AffinityPropagation                0.705632  0.222672   
DBSCAN                             0.000000  0.000000   
HDBSCAN                            1.000000  0.000000   
KMeans                             0.734719  0.386622   
OPTICS                             0.996882  0.006235   
RecursiveClustering                0.977885  0.036395   
SpectralSubspaceRandomization      1.000000  0.000000   
WardAgglomerativeClustering        0.924978  0.090899   

                              Number of Clusters Found               \
                                                median   Q25    Q75   
Model                                                                 
AffinityPropagation                               10.0  7.25  12.25   
DBSCAN                                             1.0  1.00   1.00   
HDBSCAN                                            5.0  5.00   5.00   
KMeans                                             5.0  4.25   5.50   
OPTICS                                             5.0  5.00   5.25   
RecursiveClustering                                5.0  5.00   5.25   
SpectralSubspaceRandomization                      5.0  5.00   5.00   
WardAgglomerativeClustering                        5.0  5.00   5.25   

                                  Time (s)             Memory (MB)  \
                                      mean         std        mean   
Model                                                                
AffinityPropagation             291.665205    6.030363    1070.730   
DBSCAN                          468.850985   56.704983    1331.866   
HDBSCAN                         245.753154    2.265686    1118.750   
KMeans                          228.895834   17.110452     328.976   
OPTICS                           81.335244    1.056877    1073.542   
RecursiveClustering            1223.913159  185.241066    1469.911   
SpectralSubspaceRandomization  1566.598713   63.137136     585.495   
WardAgglomerativeClustering     264.170412    5.403925    1106.073   

                                          Number of samples  ...  \
                                      std              mean  ...   
Model                                                        ...   
AffinityPropagation            640.089790             100.0  ...   
DBSCAN                         480.067282             100.0  ...   
HDBSCAN                        644.658505             100.0  ...   
KMeans                           8.565886             100.0  ...   
OPTICS                         643.367891             100.0  ...   
RecursiveClustering            561.160744             100.0  ...   
SpectralSubspaceRandomization  351.864159             100.0  ...   
WardAgglomerativeClustering    640.410057             100.0  ...   

                              Number of features         Class Separation  \
                                            mean nunique             mean   
Model                                                                       
AffinityPropagation                      10000.0       1             50.0   
DBSCAN                                   10000.0       1             50.0   
HDBSCAN                                  10000.0       1             50.0   
KMeans                                   10000.0       1             50.0   
OPTICS                                   10000.0       1             50.0   
RecursiveClustering                      10000.0       1             50.0   
SpectralSubspaceRandomization            10000.0       1             50.0   
WardAgglomerativeClustering              10000.0       1             50.0   

                                      % Random Features          \
                              nunique              mean nunique   
Model                                                             
AffinityPropagation                 1              

In [196]:
df = df_common_fill.copy()
n_samples = 1000
df = df.loc[df['n_samples'] == n_samples]
df = df.loc[df['n_features'] == 10000]
# df = df.loc[df['pct_random'] == 0.5]
df = df.loc[df['class_sep'] == 50.0]
df = df.loc[df['model_nickname'].isin(['RecursiveClustering', 'KMeans', 'HDBSCAN', 'DBSCAN', 'AffinityPropagation', 'OPTICS', 'SpectralSubspaceRandomization', 'WardAgglomerativeClustering'])]
df = df.sort_values('model_nickname')
df = df.rename(columns={'fit_model_return_elapsed_time': 'Time (s)', 'max_memory_used': 'Memory (MB)', 'n_samples': 'Number of samples', 'n_features': 'Number of features', 'model_nickname': 'Model', 'best_adjusted_rand': 'Adjusted Rand', 'best_n_clusters_': 'Number of Clusters Found', 'n_classes': 'True Number of Clusters', 'class_sep': 'Class Separation', 'pct_random': '% Random Features', 'seed_unified': 'Seed'})
df = df[['Model', 'Adjusted Rand', 'Time (s)', 'Memory (MB)', 'Number of samples', 'Number of features', 'Class Separation', '% Random Features', 'Seed', 'Number of Clusters Found', 'True Number of Clusters']]
df = df.groupby('Model').agg({'Adjusted Rand': ['mean', 'std'], 'Number of Clusters Found': ['median', ('Q25', lambda x: x.quantile(0.25)), ('Q75', lambda x: x.quantile(0.75))], 'Time (s)': ['mean', 'std'], 'Memory (MB)': ['mean', 'std'], 'Number of samples': ['mean', 'nunique'], 'Number of features': ['mean', 'nunique'], 'Class Separation': ['mean', 'nunique'], '% Random Features': ['mean', 'nunique'], 'True Number of Clusters': ['mean', 'nunique'], 'Seed': ['mean', 'nunique']})
df.to_csv(f'simulated_data_medium_{n_samples}samples.csv')
df

Adjusted Rand            \
                                       mean       std   
Model                                                   
AffinityPropagation                0.394753  0.171944   
DBSCAN                             0.000000  0.000000   
HDBSCAN                            1.000000  0.000000   
KMeans                             0.705713  0.436123   
OPTICS                             1.000000  0.000000   
RecursiveClustering                0.960138  0.058236   
SpectralSubspaceRandomization      1.000000  0.000000   
WardAgglomerativeClustering        0.999165  0.001446   

                              Number of Clusters Found               Time (s)  \
                                                median  Q25  Q75         mean   
Model                                                                           
AffinityPropagation                                3.0  2.5  5.5   212.604738   
DBSCAN                                             1.0  1.0  1.0   433.483653   
HDBSCAN                                            5.0  5.0  5.0   223.830181   
KMeans                                             5.0  3.5  5.0   194.259393   
OPTICS                                             5.0  5.0  5.0   435.720034   
RecursiveClustering                                5.0  5.0  5.5  1211.743927   
SpectralSubspaceRandomization                      5.0  5.0  5.0  4635.317239   
WardAgglomerativeClustering                        5.0  5.0  5.0   279.317189   

                                           Memory (MB)               \
                                      std         mean          std   
Model                                                                 
AffinityPropagation             43.821401  1863.784000  1419.461313   
DBSCAN                          12.986188  1807.408000   331.777796   
HDBSCAN                         16.124290  1411.584000   604.404744   
KMeans                          25.739411   725.601333   442.063026   
OPTICS                           5.135288  1349.556000   604.088144   
RecursiveClustering             99.746887  1532.300000   403.019641   
SpectralSubspaceRandomization  440.833632  1483.517333   401.519560   
WardAgglomerativeClustering     27.937345  1514.278667   427.108678   

                              Number of samples  ... Number of features  \
                                           mean  ...               mean   
Model                                            ...                      
AffinityPropagation                      1000.0  ...            10000.0   
DBSCAN                                   1000.0  ...            10000.0   
HDBSCAN                                  1000.0  ...            10000.0   
KMeans                                   1000.0  ...            10000.0   
OPTICS                                   1000.0  ...            10000.0   
RecursiveClustering                      1000.0  ...            10000.0   
SpectralSubspaceRandomization            1000.0  ...            10000.0   
WardAgglomerativeClustering              1000.0  ...            10000.0   

                                      Class Separation          \
                              nunique             mean nunique   
Model                                                            
AffinityPropagation                 1             50.0       1   
DBSCAN                              1             50.0       1   
HDBSCAN                             1             50.0       1   
KMeans                              1             50.0       1   
OPTICS                              1             50.0       1   
RecursiveClustering                 1             50.0       1   
SpectralSubspaceRandomization       1             50.0       1   
WardAgglomerativeClustering         1             50.0       1   

                              % Random Features          \
                                           mean nunique   
Model                                                    

In [197]:
df = df_common_fill.copy()
n_samples = 10000
df = df.loc[df['n_samples'] == n_samples]
df = df.loc[df['n_features'] == 10000]
# df = df.loc[df['pct_random'] == 0.5]
df = df.loc[df['class_sep'] == 50.0]
df = df.loc[df['model_nickname'].isin(['RecursiveClustering', 'KMeans', 'HDBSCAN', 'DBSCAN', 'AffinityPropagation', 'OPTICS', 'SpectralSubspaceRandomization', 'WardAgglomerativeClustering'])]
df = df.sort_values('model_nickname')
df = df.rename(columns={'fit_model_return_elapsed_time': 'Time (s)', 'max_memory_used': 'Memory (MB)', 'n_samples': 'Number of samples', 'n_features': 'Number of features', 'model_nickname': 'Model', 'best_adjusted_rand': 'Adjusted Rand', 'best_n_clusters_': 'Number of Clusters Found', 'n_classes': 'True Number of Clusters', 'class_sep': 'Class Separation', 'pct_random': '% Random Features', 'seed_unified': 'Seed'})
df = df[['Model', 'Adjusted Rand', 'Time (s)', 'Memory (MB)', 'Number of samples', 'Number of features', 'Class Separation', '% Random Features', 'Seed', 'Number of Clusters Found', 'True Number of Clusters']]
df = df.groupby('Model').agg({'Adjusted Rand': ['mean', 'std'], 'Number of Clusters Found': ['median', ('Q25', lambda x: x.quantile(0.25)), ('Q75', lambda x: x.quantile(0.75))], 'Time (s)': ['mean', 'std'], 'Memory (MB)': ['mean', 'std'], 'Number of samples': ['mean', 'nunique'], 'Number of features': ['mean', 'nunique'], 'Class Separation': ['mean', 'nunique'], '% Random Features': ['mean', 'nunique'], 'True Number of Clusters': ['mean', 'nunique'], 'Seed': ['mean', 'nunique']})
df.to_csv(f'simulated_data_medium_{n_samples}samples.csv')
df

Adjusted Rand            \
                                       mean       std   
Model                                                   
AffinityPropagation                0.087754  0.019162   
DBSCAN                             0.000000  0.000000   
HDBSCAN                                 NaN       NaN   
KMeans                             0.793508  0.333400   
OPTICS                             1.000000  0.000000   
RecursiveClustering                0.929387  0.130033   
SpectralSubspaceRandomization           NaN       NaN   
WardAgglomerativeClustering        0.740728  0.083625   

                              Number of Clusters Found                 \
                                                median    Q25     Q75   
Model                                                                   
AffinityPropagation                              112.5  93.50  133.75   
DBSCAN                                             1.0   1.00    1.00   
HDBSCAN                                            NaN    NaN     NaN   
KMeans                                             5.0   4.25    5.00   
OPTICS                                             5.0   5.00    5.00   
RecursiveClustering                                5.0   4.75    5.00   
SpectralSubspaceRandomization                      NaN    NaN     NaN   
WardAgglomerativeClustering                        4.0   3.75    4.00   

                                  Time (s)             Memory (MB)  \
                                      mean         std        mean   
Model                                                                
AffinityPropagation            5300.751929  457.882468    4447.989   
DBSCAN                         1079.975262   40.682919    3297.037   
HDBSCAN                                inf         NaN         inf   
KMeans                          563.652400   31.476729    2326.306   
OPTICS                         8207.799477  992.601355    3084.931   
RecursiveClustering            3845.434804  570.223215    3136.996   
SpectralSubspaceRandomization          inf         NaN         inf   
WardAgglomerativeClustering    7515.465591  141.255042    3099.448   

                                          Number of samples  ...  \
                                      std              mean  ...   
Model                                                        ...   
AffinityPropagation            183.783045           10000.0  ...   
DBSCAN                         295.117361           10000.0  ...   
HDBSCAN                               NaN           10000.0  ...   
KMeans                         740.989262           10000.0  ...   
OPTICS                         320.679048           10000.0  ...   
RecursiveClustering            294.056798           10000.0  ...   
SpectralSubspaceRandomization         NaN           10000.0  ...   
WardAgglomerativeClustering    320.918731           10000.0  ...   

                              Number of features         Class Separation  \
                                            mean nunique             mean   
Model                                                                       
AffinityPropagation                      10000.0       1             50.0   
DBSCAN                                   10000.0       1             50.0   
HDBSCAN                                  10000.0       1             50.0   
KMeans                                   10000.0       1             50.0   
OPTICS                                   10000.0       1             50.0   
RecursiveClustering                      10000.0       1             50.0   
SpectralSubspaceRandomization            10000.0       1             50.0   
WardAgglomerativeClustering              10000.0       1             50.0   

                                      % Random Features          \
                              nunique              mean nunique   
Model                                                             
AffinityPropagation          

# Debug and explore

In [106]:
df = df_runs_raw_parents.copy()

In [107]:
df

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,max_memory_used,mutual_info,n_clusters_,normalized_mutual_info,rand_score,silhouette,EXCEPTION,best_child_run_id,parent_run_id,raised_exception
run_uuid,,,,,,,,,,,,,,,,,,,,,
0018714386f44f95ab7141d480ce09f0,FINISHED,1737557036103,1.737557e+12,1044,eye_movements,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,422.740,0.003246,6.0,0.005303,0.358073,0.527841,NaN,756cc869acc94fed89376ae563e075a2,NaN,False
00502ff54da4466191d5fadd2ab0f945,FINISHED,1737556950053,1.737557e+12,7,audiology,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,361.448,0.078387,3.0,0.058583,0.213176,-0.124492,NaN,3d80243180a847ceb7542ac29e31edd8,NaN,False
00b6f8e09ce1419e84214bb9734a9bff,FINISHED,1737589285408,1.737590e+12,23380,cjs,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,491.640,0.271687,51.0,0.125215,0.739143,0.441531,NaN,f7b70e6394ce4793bd5d1924df8d7cf0,NaN,False
00b7b74b49c2418e8cd9108326d76f54,FINISHED,1737504019181,1.737512e+12,46335,primary-tumor_clean,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,541.000,2.169609,216.0,0.563600,0.891047,0.359055,NaN,f2af19c022474cc587b3a26c1c1518b7,NaN,False
0200fd7653474a6ab77e2fa4085e294f,FINISHED,1737617015722,1.737619e+12,16,mfeat-karhunen,maximize,adjusted_rand,OPTICS,NaN,NaN,...,1005.424,0.028445,4.0,0.023843,0.120347,-0.058768,NaN,e1c3ab245a3e47d0813cd19cb56802d0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fb946510558746d29d122b87ee9869da,FINISHED,1737617084498,1.737620e+12,30,page-blocks,maximize,adjusted_rand,OPTICS,NaN,NaN,...,1005.424,0.081613,80.0,0.099852,0.634365,-0.584304,NaN,6578c7710d3d445eb88dd432d6c88b91,NaN,False
fd61b8a547c54e11b898653ebc8769db,FINISHED,1737557019786,1.737557e+12,40984,segment,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,410.420,1.356621,33.0,0.598839,0.856482,0.196516,NaN,d81f797b53d142e68e400b8143cb2586,NaN,False
fdc6bb9a534743c7be6ce026fd67252a,FINISHED,1737556999587,1.737557e+12,61,iris,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,7826.080,0.638874,3.0,0.685822,0.772975,0.507009,NaN,abfd2a33adaf48689df0ef0a86e211b6,NaN,False


In [108]:
df = df.loc[df['status'] == 'RUNNING']

In [109]:
df

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,max_memory_used,mutual_info,n_clusters_,normalized_mutual_info,rand_score,silhouette,EXCEPTION,best_child_run_id,parent_run_id,raised_exception
run_uuid,,,,,,,,,,,,,,,,,,,,,
0289c581d4a74c6998008fd8c6a23a8b,RUNNING,1737633122174,NaN,41168,NaN,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,0.145176,25.0,0.073103,0.630174,-0.003434,NaN,NaN,NaN,NaN
0417c1059c264cdf8399c22c311b9876,RUNNING,1737514361699,NaN,41168,NaN,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
043d0cbd906747819040f8979d7f40d1,RUNNING,1737626372810,NaN,40996,NaN,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
08ca8fb95ff84adcb2b8c6567a29cc34,RUNNING,1737626671949,NaN,41168,NaN,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,0.145176,25.0,0.073103,0.630174,-0.003434,NaN,NaN,NaN,NaN
16a5d69973d3475eae567d57d9241d3d,RUNNING,1737516197385,NaN,41027,NaN,maximize,adjusted_rand,AffinityPropagation,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1798d892c9b54192ac736b8013d9229e,RUNNING,1737617191351,NaN,1596,NaN,maximize,adjusted_rand,OPTICS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1e3053ee2da046f8aaf7a0e0c70ba9dc,RUNNING,1737589047902,NaN,184,NaN,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,NaN,0.000000,1.0,0.000000,0.104322,-1.000000,NaN,NaN,NaN,NaN
230e310a3ff640f6865d1bda85845caf,RUNNING,1737504059995,NaN,1501,NaN,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,NaN,0.949910,64.0,0.335039,0.880739,-0.074361,NaN,NaN,NaN,NaN
2490402188d94a7ca9f499eee32a52d3,RUNNING,1737590230603,NaN,1501,NaN,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,NaN,0.272477,7.0,0.140823,0.686752,-0.005928,NaN,NaN,NaN,NaN


In [110]:
parent_run_ids = list(df.index)

In [111]:
parent_run_ids

['0289c581d4a74c6998008fd8c6a23a8b',
 '0417c1059c264cdf8399c22c311b9876',
 '043d0cbd906747819040f8979d7f40d1',
 '08ca8fb95ff84adcb2b8c6567a29cc34',
 '16a5d69973d3475eae567d57d9241d3d',
 '1798d892c9b54192ac736b8013d9229e',
 '1e3053ee2da046f8aaf7a0e0c70ba9dc',
 '230e310a3ff640f6865d1bda85845caf',
 '2490402188d94a7ca9f499eee32a52d3',
 '2545cfaa4ce44d4ba25a4c57945f8f4a',
 '26ab3cc84c4a420b8857b1a95ed6f767',
 '2ec4852314ed431f96d3841991fdc743',
 '338ba54bce1345a2a924b657bd5993bf',
 '3692c78f34fd4c3c885c21c7a9071eaa',
 '3eb48652d7dd4c87ac58ddd07b3c5175',
 '4ea354c749a844fea7fa84bbb49cb447',
 '54433df700ed4a78801c8425094c326f',
 '571eb50796254ba09de64dc30acdebc1',
 '6a48f613362447c78e3359b5c9dcc2f3',
 '6cc3d2d840764cbea8c488d14e14b443',
 '6e847cd448bd4b8184b831772de744bd',
 '71d0891f6ac041e38d423fe3328f130e',
 '7239cc6680ec46a880e51cd9e02c3b62',
 '72528d7e02ad4ba1b51ba2ca3335ae83',
 '72cbb804b0d44f4e83b8d6e723bde1d2',
 '7354437c98ce4de9af3822771725e288',
 '7e33df0102fb40968e05fb8ce3e3c909',
 

In [112]:
df = df_runs_raw.copy()
df = df.loc[df['parent_run_id'].isin(parent_run_ids)]

In [113]:
child_run_ids = list(df.index)

In [114]:
runs_to_delete = parent_run_ids + child_run_ids

In [115]:
runs_to_delete

['0289c581d4a74c6998008fd8c6a23a8b',
 '0417c1059c264cdf8399c22c311b9876',
 '043d0cbd906747819040f8979d7f40d1',
 '08ca8fb95ff84adcb2b8c6567a29cc34',
 '16a5d69973d3475eae567d57d9241d3d',
 '1798d892c9b54192ac736b8013d9229e',
 '1e3053ee2da046f8aaf7a0e0c70ba9dc',
 '230e310a3ff640f6865d1bda85845caf',
 '2490402188d94a7ca9f499eee32a52d3',
 '2545cfaa4ce44d4ba25a4c57945f8f4a',
 '26ab3cc84c4a420b8857b1a95ed6f767',
 '2ec4852314ed431f96d3841991fdc743',
 '338ba54bce1345a2a924b657bd5993bf',
 '3692c78f34fd4c3c885c21c7a9071eaa',
 '3eb48652d7dd4c87ac58ddd07b3c5175',
 '4ea354c749a844fea7fa84bbb49cb447',
 '54433df700ed4a78801c8425094c326f',
 '571eb50796254ba09de64dc30acdebc1',
 '6a48f613362447c78e3359b5c9dcc2f3',
 '6cc3d2d840764cbea8c488d14e14b443',
 '6e847cd448bd4b8184b831772de744bd',
 '71d0891f6ac041e38d423fe3328f130e',
 '7239cc6680ec46a880e51cd9e02c3b62',
 '72528d7e02ad4ba1b51ba2ca3335ae83',
 '72cbb804b0d44f4e83b8d6e723bde1d2',
 '7354437c98ce4de9af3822771725e288',
 '7e33df0102fb40968e05fb8ce3e3c909',
 

In [116]:
run_uuid_query = [f"'{run_id}'" for run_id in runs_to_delete]
run_uuid_query = ', '.join(run_uuid_query)

In [117]:
query = f"""
UPDATE runs
SET lifecycle_stage = 'deleted'
WHERE run_uuid IN ({run_uuid_query}) 
"""
with engine.begin() as conn:
    conn.execute(text(query))

for i, row in df_runs_raw.iterrows():
    run_id = row.run_id
    model_name = row['params.model_name']
    with mlflow.start_run(run_id) as run:
        mlflow.log_param('model_nickname', model_name)    